Data used in this study is from the public dataset: 

Imaging-psychiatry challenge: predicting autism \
A data challenge on Autism Spectrum Disorder detection
Roberto Toro (Institut Pasteur), Nicolas Traut (Institut Pasteur), Anita Beggiato (Institut Pasteur), Katja Heuer (Institut Pasteur),
Gael Varoquaux (Inria, Parietal), Alex Gramfort (Inria, Parietal), Balazs Kegl (LAL),
Guillaume Lemaitre (CDS), Alexandre Boucaud (CDS), and Joris van den Bossche (CDS)

available at: https://paris-saclay-cds.github.io/autism_challenge/

In [1]:
from problem import get_train_data
data_train, labels_train = get_train_data()

In [2]:
!pip install numpy==1.21.6
!pip install neuroHarmonize==2.4.5
import os
import numpy as np
import pandas as pd
import random
from neuroHarmonize import harmonizationLearn, harmonizationApply, loadHarmonizationModel

import os
import numpy as np
import pandas as pd
import joblib
import argparse
import glob
import shutil
import hashlib
import zipfile
try:
    from urllib.request import urlretrieve
except ImportError:
    from urllib import urlretrieve

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer


!pip install nilearn
from nilearn.connectome import ConnectivityMeasure
from nilearn import plotting

ERROR: Ignored the following versions that require a different python version: 1.21.2 Requires-Python >=3.7,<3.11; 1.21.3 Requires-Python >=3.7,<3.11; 1.21.4 Requires-Python >=3.7,<3.11; 1.21.5 Requires-Python >=3.7,<3.11; 1.21.6 Requires-Python >=3.7,<3.11
ERROR: Could not find a version that satisfies the requirement numpy==1.21.6 (from versions: 1.3.0, 1.4.1, 1.5.0, 1.5.1, 1.6.0, 1.6.1, 1.6.2, 1.7.0, 1.7.1, 1.7.2, 1.8.0, 1.8.1, 1.8.2, 1.9.0, 1.9.1, 1.9.2, 1.9.3, 1.10.0.post2, 1.10.1, 1.10.2, 1.10.4, 1.11.0, 1.11.1, 1.11.2, 1.11.3, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 1.13.3, 1.14.0, 1.14.1, 1.14.2, 1.14.3, 1.14.4, 1.14.5, 1.14.6, 1.15.0, 1.15.1, 1.15.2, 1.15.3, 1.15.4, 1.16.0, 1.16.1, 1.16.2, 1.16.3, 1.16.4, 1.16.5, 1.16.6, 1.17.0, 1.17.1, 1.17.2, 1.17.3, 1.17.4, 1.17.5, 1.18.0, 1.18.1, 1.18.2, 1.18.3, 1.18.4, 1.18.5, 1.19.0, 1.19.1, 1.19.2, 1.19.3, 1.19.4, 1.19.5, 1.20.0, 1.20.1, 1.20.2, 1.20.3, 1.21.0, 1.21.1, 1.22.0, 1.22.1, 1.22.2, 1.22.3, 1.22.4, 1.23.0rc1, 1.23.0rc2, 1.23.0rc3, 1.2

# Helper

In [3]:
def _read_data(path, filename):

    subject_id = pd.read_csv(os.path.join(path, 'data', filename), header=None)
    # read the list of the subjects
    df_participants = pd.read_csv(os.path.join(path, 'data','participants.csv'),
                                  index_col=0)
    df_participants.columns = ['participants_' + col
                               for col in df_participants.columns]

    # load the structural and functional MRI data
    df_anatomy = pd.read_csv(os.path.join(path, 'data', 'anatomy.csv'),
                             index_col=0)
    df_anatomy.columns = ['anatomy_' + col
                          for col in df_anatomy.columns]
    df_fmri = pd.read_csv(os.path.join(path, 'data', 'fmri_filename.csv'),
                          index_col=0)
    df_fmri.columns = ['fmri_' + col
                       for col in df_fmri.columns]

    # load the QC for structural and functional MRI data
    df_anatomy_qc = pd.read_csv(os.path.join(path, 'data', 'anatomy_qc.csv'),
                                index_col=0)
    df_fmri_qc = pd.read_csv(os.path.join(path, 'data', 'fmri_qc.csv'),
                             index_col=0)
    df_fmri_tr = pd.read_csv(os.path.join(path, 'data','fmri_repetition_time.csv'),
                             index_col=0)

    # rename the columns for the QC to have distinct names
    df_anatomy_qc = df_anatomy_qc.rename(columns={"select": "anatomy_select"})
    df_fmri_qc = df_fmri_qc.rename(columns={"select": "fmri_select"})

    X = pd.concat([df_participants, df_anatomy, df_anatomy_qc, df_fmri,
                   df_fmri_qc, df_fmri_tr], axis=1)
    X = X.loc[subject_id[0]]
    y = X['participants_asd']
    X = X.drop('participants_asd', axis=1)
    return X, y.values

def get_train_data(path='.'):
    filename = 'train.csv'
    return _read_data(path, filename)

def get_test_data(path='.'):
    filename = 'test.csv'
    return _read_data(path, filename)

In [4]:
ATLAS = ('craddock_scorr_mean', 'power_2011')
ARCHIVE = {atlas: 'https://zenodo.org/record/3625740/files/{}.zip'
           .format(atlas) for atlas in ATLAS}
CHECKSUM = {
    'craddock_scorr_mean':
    '634e0bb07beaae033a0f1615aa885ba4cb67788d4a6e472fd432a1226e01b49b',
    'power_2011':
    'd1e3cd8eaa867079fe6b24dfaee08bd3b2d9e0ebbd806a2a982db5407328990a'}

# ATLAS = ('basc064', 'basc122', 'basc197', 'craddock_scorr_mean',
#          'harvard_oxford_cort_prob_2mm', 'msdl', 'power_2011')
# ARCHIVE = {atlas: 'https://storage.ramp.studio/autism/{}.zip'.format(atlas)
#            for atlas in ATLAS}
# CHECKSUM = {
#     'basc064':
#     '75eb5ee72344d11f056551310a470d00227fac3e87b7205196f77042fcd434d0',
#     'basc122':
#     '2d0d2c2338f9114877a0a1eb695e73f04fc664065d1fb75cff8d59f6516b0ec7',
#     'basc197':
#     '68135bb8e89b5b3653e843745d8e5d0e92876a5536654eaeb9729c9a52ab00e9',
#     'craddock_scorr_mean':
#     '634e0bb07beaae033a0f1615aa885ba4cb67788d4a6e472fd432a1226e01b49b',
#     'harvard_oxford_cort_prob_2mm':
#     '638559dc4c7de25575edc02e58404c3f2600556239888cbd2e5887316def0e74',
#     'msdl':
#     'fd241bd66183d5fc7bdf9a115d7aeb9a5fecff5801cd15a4e5aed72612916a97',
#     'power_2011':
#     'd1e3cd8eaa867079fe6b24dfaee08bd3b2d9e0ebbd806a2a982db5407328990a'}

def _sha256(path):
    """Calculate the sha256 hash of the file at path."""
    sha256hash = hashlib.sha256()
    chunk_size = 8192
    with open(path, "rb") as f:
        while True:
            buffer = f.read(chunk_size)
            if not buffer:
                break
            sha256hash.update(buffer)
    return sha256hash.hexdigest()

def _check_and_unzip(output_file, atlas, atlas_directory):
    checksum_download = _sha256(output_file)
    if checksum_download != CHECKSUM[atlas]:
        os.remove(output_file)
        raise IOError('The file downloaded was corrupted. Try again '
                      'to execute this script.')
    print('Decompressing the archive ...')
    zip_ref = zipfile.ZipFile(output_file, 'r')
    zip_ref.extractall(atlas_directory)
    zip_ref.close()


def _download_fmri_data(atlas):
    print('Downloading the data from {} ...'.format(ARCHIVE[atlas]))
    output_file = os.path.abspath(
        os.path.join('.', 'data', 'fmri', atlas + '.zip'))
    urlretrieve(ARCHIVE[atlas], filename=output_file)
    atlas_directory = os.path.abspath(os.path.join('.', 'data', 'fmri'))
    _check_and_unzip(output_file, atlas, atlas_directory)


def _check_integrity_atlas(atlas):
    # check that the folder is existing
    atlas_directory = os.path.abspath(os.path.join('.', 'data', 'fmri', atlas))
    if os.path.isdir(atlas_directory):
        # compute the hash of the current data present on the disk
        filenames_atlas_current = np.array(
            glob.glob(os.path.join(atlas_directory, '*', '*', '*')),
            dtype=object)
        filenames_atlas_current.sort()
        current_hash = joblib.hash(filenames_atlas_current)

        # compute the expected hash from the data set which we provide
        filenames_atlas_expected = pd.read_csv(
            os.path.abspath(os.path.join('.', 'data', 'fmri_filename.csv')),
            index_col=0)[atlas].values
        for idx in range(filenames_atlas_expected.size):
            filenames_atlas_expected[idx] = os.path.abspath(
                filenames_atlas_expected[idx])
        filenames_atlas_expected.sort()
        expected_hash = joblib.hash(filenames_atlas_expected)

        if current_hash == expected_hash:
            return

        shutil.rmtree(atlas_directory)

    _download_fmri_data(atlas)

def fetch_fmri_time_series(atlas='all'):
    if atlas == 'all':
        for single_atlas in ATLAS:
            _check_integrity_atlas(single_atlas)
    elif atlas in ATLAS:
        _check_integrity_atlas(atlas)
    else:
        raise ValueError("'atlas' should be one of {}. Got {} instead."
                         .format(ATLAS, atlas))
    print('Downloading completed.')

def load_fmri(fmri_filenames):
    """Load time-series extracted from the fMRI using a specific atlas."""
    data = []
    for i, subject_filename in enumerate(fmri_filenames):
        subject_data = pd.read_csv(subject_filename, header=None).values

        data.append(subject_data)
    return np.array(data, dtype = object)


# Download data -- Craddock atlas


In [6]:
# get the fmri file paths
data_train, labels_train = get_train_data(path='.')
data_test, labels_test = get_test_data(path='.')

# get functional connectivity features
atlas_want = 'craddock_scorr_mean' # 'power_2011'
atlas_name = 'fmri_'+atlas_want
fetch_fmri_time_series(atlas=atlas_want)

In [7]:
fmri_data_train = load_fmri(data_train[atlas_name])
fmri_data_test = load_fmri(data_test[atlas_name])
fmri_data = np.hstack((fmri_data_train,fmri_data_test)) #concat train and test together
# compute correlation matrix
correlation_measure = ConnectivityMeasure(kind='correlation', vectorize=True)
cor_data = correlation_measure.fit_transform(fmri_data)
# get autism labels
autism_labels = np.hstack((labels_train,labels_test))
# get quality check info
qc = np.hstack((data_train['fmri_select'], data_test['fmri_select']))
# get site sex and age info
site = np.hstack((data_train['participants_site'], data_test['participants_site']))
sex = np.hstack((data_train['participants_sex'], data_test['participants_sex']))
age = np.hstack((data_train['participants_age'], data_test['participants_age']))

/opt/homebrew/lib/python3.11/site-packages/nilearn/connectome/connectivity_matrices.py:509: FutureWarning: The default strategy for standardize is currently 'zscore' which incorrectly uses population std to calculate sample zscores. The new strategy 'zscore_sample' corrects this behavior by using the sample std. In release 0.13, the default strategy will be replaced by the new strategy and the 'zscore' option will be removed. Please use 'zscore_sample' instead.
  covariances_std = [
/opt/homebrew/lib/python3.11/site-packages/nilearn/connectome/connectivity_matrices.py:509: FutureWarning: The default strategy for standardize is currently 'zscore' which incorrectly uses population std to calculate sample zscores. The new strategy 'zscore_sample' corrects this behavior by using the sample std. In release 0.13, the default strategy will be replaced by the new strategy and the 'zscore' option will be removed. Please use 'zscore_sample' instead.
  covariances_std = [
/opt/homebrew/lib/python

In [8]:
# save data
np.save(os.path.join('.', 'data', 'fmri',atlas_name+'_cor'), cor_data)
np.save(os.path.join('.', 'data', 'fmri','autism_labels'), autism_labels)
np.save(os.path.join('.', 'data', 'fmri','autism_qc'), qc)
np.save(os.path.join('.', 'data', 'fmri','autism_site'), site)
np.save(os.path.join('.', 'data', 'fmri','autism_sex'), sex)
np.save(os.path.join('.', 'data', 'fmri','autism_age'), age)
print('Data saved.')

Data saved.


# Download data -- Power Atlas

In [9]:
# get the fmri file paths
data_train, labels_train = get_train_data(path='.')
data_test, labels_test = get_test_data(path='.')

In [10]:
# get functional connectivity features
atlas_want = 'power_2011' # 'craddock_scorr_mean'
atlas_name = 'fmri_'+atlas_want
fetch_fmri_time_series(atlas=atlas_want)

In [11]:
fmri_data_train = load_fmri(data_train[atlas_name])
fmri_data_test = load_fmri(data_test[atlas_name])
fmri_data = np.hstack((fmri_data_train,fmri_data_test)) # concat train and test together
# computer correlation matrix
correlation_measure = ConnectivityMeasure(kind='correlation', vectorize=True)
cor_data = correlation_measure.fit_transform(fmri_data)
# get autism labels
autism_labels = np.hstack((labels_train,labels_test))
# get quality check info
qc = np.hstack((data_train['fmri_select'], data_test['fmri_select']))
# get site sex and age info
site = np.hstack((data_train['participants_site'], data_test['participants_site']))
sex = np.hstack((data_train['participants_sex'], data_test['participants_sex']))
age = np.hstack((data_train['participants_age'], data_test['participants_age']))

/opt/homebrew/lib/python3.11/site-packages/nilearn/connectome/connectivity_matrices.py:509: FutureWarning: The default strategy for standardize is currently 'zscore' which incorrectly uses population std to calculate sample zscores. The new strategy 'zscore_sample' corrects this behavior by using the sample std. In release 0.13, the default strategy will be replaced by the new strategy and the 'zscore' option will be removed. Please use 'zscore_sample' instead.
  covariances_std = [
/opt/homebrew/lib/python3.11/site-packages/nilearn/connectome/connectivity_matrices.py:509: FutureWarning: The default strategy for standardize is currently 'zscore' which incorrectly uses population std to calculate sample zscores. The new strategy 'zscore_sample' corrects this behavior by using the sample std. In release 0.13, the default strategy will be replaced by the new strategy and the 'zscore' option will be removed. Please use 'zscore_sample' instead.
  covariances_std = [
/opt/homebrew/lib/python

In [12]:
# save data
np.save(os.path.join('.', 'data', 'fmri',atlas_name+'_cor'), cor_data)
np.save(os.path.join('.', 'data', 'fmri','autism_labels'), autism_labels)
np.save(os.path.join('.', 'data', 'fmri','autism_qc'), qc)
np.save(os.path.join('.', 'data', 'fmri','autism_site'), site)
np.save(os.path.join('.', 'data', 'fmri','autism_sex'), sex)
np.save(os.path.join('.', 'data', 'fmri','autism_age'), age)
print('Data saved.')

Data saved.


# QC and Combat -- Craddock

In [14]:
atlas_want = 'craddock_scorr_mean'
datapath = './data/fmri'

# load correlation data and subject info
label = np.load(os.path.join(datapath, 'autism_labels.npy'))
data = np.load(os.path.join(datapath, 'fmri_'+atlas_want+'_cor.npy'))
qc = np.load(os.path.join(datapath, 'autism_qc.npy'))
site = np.load(os.path.join(datapath, 'autism_site.npy'))
age = np.load(os.path.join(datapath, 'autism_age.npy'))
sex = np.load(os.path.join(datapath, 'autism_sex.npy'), allow_pickle=True)
sex[sex =='F'] = 0
sex[sex =='M'] = 1
sex = sex.astype(str).astype(int)

# remove subjects of bad imaging quality
data_qc = data[qc ==1,:]
label_qc = label[qc ==1]
site_qc = site[qc ==1]
age_qc = age[qc ==1]
sex_qc = sex[qc ==1]

In [15]:
# set a seed
SEED = 10
random.seed(SEED)
# split the test site
test_i = np.array(random.sample(range(len(data_qc)), k=int(len(data_qc)*0.2)))

train_val_mask = np.ones(len(data_qc), bool)
train_val_mask[test_i] = 0

test_data = data_qc[test_i,:]
test_sex = sex_qc[test_i]
test_age = age_qc[test_i]
test_label = label_qc[test_i]
test_site = site_qc[test_i]

train_val_data = data_qc[train_val_mask,:]
train_val_sex = sex_qc[train_val_mask]
train_val_age = age_qc[train_val_mask]
train_val_label = label_qc[train_val_mask]
train_val_site = site_qc[train_val_mask]

In [16]:
np.savetxt(os.path.join(datapath, atlas_want+'_trainval_data_qc.csv'), train_val_data, delimiter=',')
np.savetxt(os.path.join(datapath, atlas_want+'_test_data_qc.csv'), test_data, delimiter=',')

# save the processed data and subject info (after quality check without combat)
np.save(os.path.join(datapath, atlas_want+'_trainval_data_qc'), train_val_data)
np.save(os.path.join(datapath, atlas_want+'_test_data_qc'), test_data)

In [17]:
# combat
# save the covariates
train_val_covars = pd.DataFrame(data = np.hstack((train_val_site.reshape(-1,1).astype(str),
                                                  train_val_age.reshape(-1,1),
                                                  train_val_sex.reshape(-1,1))),
                                columns = ['SITE','AGE','SEX_M'])

test_covars = pd.DataFrame(data = np.hstack((test_site.reshape(-1,1).astype(str),
                                             test_age.reshape(-1,1),
                                             test_sex.reshape(-1,1))),
                           columns = ['SITE','AGE','SEX_M'])

train_val_covars.to_csv(os.path.join(datapath, 'trainval_covars_adj_qc.csv'), index = False)
test_covars.to_csv(os.path.join(datapath, 'test_covars_adj_qc.csv'), index = False)

# run combact on train+val dataset
''' 
Note that inclusion of the smooth_terms argument uses GAM to adjust for the age and site effects
Not using the smooth_terms argument instead results in linear adjustment of the covariates. 
When we attempt to include the smooth_terms argument we obtain a Perfect separation error.
'''
combat_model, train_val_adj_qc = harmonizationLearn(train_val_data, train_val_covars,
                                                    eb=True , seed= SEED,
                                                    #smooth_terms=["AGE", "SEX_M"]
                                                    )

# apply combat_model to the test dataset
test_data_adj_qc = harmonizationApply(test_data, test_covars, combat_model)

In [18]:
# save the processed data and subject info (after quality check and combat)
np.save(os.path.join(datapath, atlas_want+'_trainval_data_adj_qc'), train_val_adj_qc) # '_adj' means after combat
np.save(os.path.join(datapath, 'trainval_label_adj_qc'), train_val_label)

np.save(os.path.join(datapath, atlas_want+'_test_data_adj_qc'), test_data_adj_qc)
np.save(os.path.join(datapath, 'test_label_adj_qc'), test_label)

# QC and Combat -- Power

In [23]:
atlas_want = 'power_2011'
datapath = './data/fmri'

# load correlation data and subject info
label = np.load(os.path.join(datapath, 'autism_labels.npy'))
data = np.load(os.path.join(datapath, 'fmri_'+atlas_want+'_cor.npy'))
qc = np.load(os.path.join(datapath, 'autism_qc.npy'))
site = np.load(os.path.join(datapath, 'autism_site.npy'))
age = np.load(os.path.join(datapath, 'autism_age.npy'))
sex = np.load(os.path.join(datapath, 'autism_sex.npy'), allow_pickle=True)
sex[sex =='F'] = 0
sex[sex =='M'] = 1
sex = sex.astype(str).astype(int)

# remove subjects of bad imaging quality
data_qc = data[qc ==1,:]
label_qc = label[qc ==1]
site_qc = site[qc ==1]
age_qc = age[qc ==1]
sex_qc = sex[qc ==1]

# set a seed
SEED = 10
random.seed(SEED)
# split the test site
test_i = np.array(random.sample(range(len(data_qc)), k=int(len(data_qc)*0.2)))

train_val_mask = np.ones(len(data_qc), bool)
train_val_mask[test_i] = 0

test_data = data_qc[test_i,:]
test_sex = sex_qc[test_i]
test_age = age_qc[test_i]
test_label = label_qc[test_i]
test_site = site_qc[test_i]

train_val_data = data_qc[train_val_mask,:]
train_val_sex = sex_qc[train_val_mask]
train_val_age = age_qc[train_val_mask]
train_val_label = label_qc[train_val_mask]
train_val_site = site_qc[train_val_mask]

np.savetxt(os.path.join(datapath, atlas_want+'_trainval_data_qc.csv'), train_val_data, delimiter=',')
np.savetxt(os.path.join(datapath, atlas_want+'_test_data_qc.csv'), test_data, delimiter=',')

# save the processed data and subject info (after quality check without combat)
np.save(os.path.join(datapath, atlas_want+'_trainval_data_qc'), train_val_data)
np.save(os.path.join(datapath, atlas_want+'_test_data_qc'), test_data)

# combat
# save the covariates
train_val_covars = pd.DataFrame(data = np.hstack((train_val_site.reshape(-1,1).astype(str),
                                                  train_val_age.reshape(-1,1),
                                                  train_val_sex.reshape(-1,1))),
                                columns = ['SITE','AGE','SEX_M'])

test_covars = pd.DataFrame(data = np.hstack((test_site.reshape(-1,1).astype(str),
                                             test_age.reshape(-1,1),
                                             test_sex.reshape(-1,1))),
                           columns = ['SITE','AGE','SEX_M'])

train_val_covars.to_csv(os.path.join(datapath, 'trainval_covars_adj_qc.csv'), index = False)
test_covars.to_csv(os.path.join(datapath, 'test_covars_adj_qc.csv'), index = False)

# run combact on train+val dataset
''' 
Note that inclusion of the smooth_terms argument uses GAM to adjust for the age and site effects
Not using the smooth_terms argument instead results in linear adjustment of the covariates. 
When we attempt to include the smooth_terms argument we obtain a Perfect separation error.
'''
combat_model, train_val_adj_qc = harmonizationLearn(train_val_data, train_val_covars,
                                                    eb=True , seed= SEED,
                                                    #smooth_terms=["AGE", "SEX_M"]
                                                    )

# apply combat_model to the test dataset
test_data_adj_qc = harmonizationApply(test_data, test_covars, combat_model)

# save the processed data and subject info (after quality check and combat)
np.save(os.path.join(datapath, atlas_want+'_trainval_data_adj_qc'), train_val_adj_qc) # '_adj' means after combat
np.save(os.path.join(datapath, 'trainval_label_adj_qc'), train_val_label)

np.save(os.path.join(datapath, atlas_want+'_test_data_adj_qc'), test_data_adj_qc)
np.save(os.path.join(datapath, 'test_label_adj_qc'), test_label)

# Demographics

In [24]:
qc_adj_info = pd.concat([train_val_covars, test_covars]).astype(float)
qc_adj_info['ASD'] = np.hstack((train_val_label, test_label)).astype(int)

In [25]:
demo_by_site = pd.DataFrame(columns = ['Site', 'NumberSubject',
                                       'NumberFemale','NumberMale',
                                       'MeanAge','StdAge','MinAge','MaxAge',
                                       'Ctrl','ASD'])

for site in pd.unique(qc_adj_info['SITE']):
  this_site_data = qc_adj_info[qc_adj_info['SITE'] == site]
  this_site_info = [site, len(this_site_data),
                    sum(this_site_data['SEX_M'] == 1), sum(this_site_data['SEX_M'] == 0),
                    np.mean(this_site_data['AGE']), np.std(this_site_data['AGE']),
                    min(this_site_data['AGE']), max(this_site_data['AGE']),
                    sum(this_site_data['ASD'] == 0), sum(this_site_data['ASD'] == 1)
                    ]
  demo_by_site.loc[site] = this_site_info

demo_by_site['Site'] = demo_by_site['Site'].astype(int)
demo_by_site = demo_by_site.sort_values(by=['Site'], ascending=True)
demo_by_site = demo_by_site.reset_index(drop=True)

demo_by_site.to_csv(os.path.join(datapath,'demo_by_site.csv'), encoding='utf-8', index=False)

## Subject Data Info

In [26]:
datainfo = pd.concat([data_train, data_test]) #data.shape
datainfo['label'] = np.hstack((labels_train, labels_test))
subj_datainfo = datainfo[['participants_site', 'participants_sex',
                          'participants_age','fmri_power_2011','fmri_select',
                          'repetition_time','label']]
subj_datainfo

,participants_site,participants_sex,participants_age,fmri_power_2011,fmri_select,repetition_time,label
subject_id,,,,,,,
1932355398536124106,5,F,9.301370,./data/fmri/power_2011/1932355398536124106/run...,1,2.5,0
5174041730092253771,19,M,29.000000,./data/fmri/power_2011/5174041730092253771/run...,1,3.0,0
10219322676643534800,19,F,45.000000,./data/fmri/power_2011/10219322676643534800/ru...,1,3.0,1
10645466564919190227,5,F,9.216438,./data/fmri/power_2011/10645466564919190227/ru...,1,2.5,1
14512541342641936232,28,M,15.050000,./data/fmri/power_2011/14512541342641936232/ru...,1,1.0,1
...,...,...,...,...,...,...,...
7223236323395509661,33,M,12.525700,./data/fmri/power_2011/7223236323395509661/run...,1,2.0,0
3369213453014636420,7,F,34.760000,./data/fmri/power_2011/3369213453014636420/run...,1,2.0,1
1711749423820760510,8,F,8.844000,./data/fmri/power_2011/1711749423820760510/run...,1,2.0,1


In [27]:
site_demo = pd.DataFrame(columns = ['Site',
                                    'NumberSubject', 'NumberFemale','NumberMale',
                                    'MeanAge','StdAge','MinAge','MaxAge',
                                    'Ctrl','ASD',
                                    'Reject','Accept','AcceptwReserve'])

for site in pd.unique(subj_datainfo['participants_site']):
  this_site_data = subj_datainfo[subj_datainfo['participants_site'] == site]
  this_site_info = [site,
                    len(this_site_data), sum(this_site_data['participants_sex'] == 'F'), sum(this_site_data['participants_sex'] == 'M'),
                    np.mean(this_site_data['participants_age']), np.std(this_site_data['participants_age']),
                    min(this_site_data['participants_age']),max(this_site_data['participants_age']),
                    sum(this_site_data['label'] == 0), sum(this_site_data['label'] == 1),
                    sum((this_site_data['fmri_select'] == 0)) + sum((this_site_data['fmri_select'] == 3)),
                    sum(this_site_data['fmri_select'] == 1),sum(this_site_data['fmri_select'] == 2)
                    ]
  site_demo.loc[site] = this_site_info

site_demo['Site'] = site_demo['Site'].astype(int)
site_demo = site_demo.sort_values(by=['Site'], ascending=True)

site_demo.to_csv(os.path.join(datapath,'site_demo.csv'), encoding='utf-8', index=False)

In [28]:
site_demo

,Site,NumberSubject,NumberFemale,NumberMale,MeanAge,StdAge,MinAge,MaxAge,Ctrl,ASD,Reject,Accept,AcceptwReserve
0,0,45.0,0.0,45.0,39.955556,14.156201,18.000000,62.000000,23.0,22.0,6.0,39.0,0.0
1,1,32.0,7.0,25.0,7.928217,0.974014,6.329911,10.650240,15.0,17.0,13.0,19.0,0.0
2,2,35.0,11.0,24.0,10.429667,1.729423,8.063014,13.882192,21.0,14.0,1.0,34.0,0.0
3,3,32.0,18.0,14.0,23.127812,11.828478,7.556000,46.600000,23.0,9.0,5.0,27.0,0.0
4,4,17.0,6.0,11.0,26.764706,10.008992,17.000000,54.000000,11.0,6.0,1.0,16.0,0.0
5,5,102.0,38.0,64.0,10.431883,1.245978,8.068493,12.986301,76.0,26.0,5.0,97.0,0.0
6,6,21.0,0.0,21.0,23.380952,3.810714,18.000000,33.000000,0.0,21.0,0.0,21.0,0.0
7,7,48.0,5.0,43.0,10.110917,5.978872,5.220000,34.760000,16.0,32.0,0.0,48.0,0.0
8,8,18.0,3.0,15.0,6.686278,0.980295,5.128000,8.844000,0.0,18.0,6.0,12.0,0.0
9,9,55.0,22.0,33.0,11.472727,2.025784,8.000000,15.000000,29.0,26.0,1.0,54.0,0.0
